In [4]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [15]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

## Wczytywanie danych

In [2]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car/"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [3]:
ls

car.h5  data/  day2_visualisation.ipynb  LICENSE  README.md


In [0]:
df = pd.read_hdf('data/car.h5')

In [5]:
df.shape

(106494, 155)

In [6]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

## Dummy model

In [9]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [17]:
feats =['car_id']
x=df[feats].values
y= df['price_value'].values

model=DummyRegressor()
model.fit(x, y)
y_pred=model.predict(x)

mae(y,y_pred)

39465.934630440985

In [18]:
#które kolumny zawierają price
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [20]:
#ile jes cen w jakiej walucie
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [0]:
#usuwanie danych w EURO
df=df[ df['price_currency'] !='EUR']

In [22]:
df.shape

(106290, 155)

## Features

array([-1, -1, -1, ..., -1, -1, -1])

In [0]:
#feat=dynamiczna nazwa poszczególnej kolumny
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):continue #jeśli wartości jesst listą to pomiń\

  factorized_values=df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat]=factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [50]:
cat_feats=[x for x in df.columns if SUFFIX_CAT in x]
cat_feats=[x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [52]:
x = df[cat_feats].values
y=df['price_value'].values

model= DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19695.13091100928

In [53]:
m= DecisionTreeRegressor(max_depth=5)
m.fit(x, y)

inp=  PermutationImportance(m, random_state=0).fit(x,y)
eli5.show_weights(inp, feature_names=cat_feats)

Weight,Feature
0.2570 ± 0.0084,param_napęd__cat
0.2033 ± 0.0037,param_stan__cat
0.1957 ± 0.0021,param_faktura-vat__cat
0.1447 ± 0.0075,param_rok-produkcji__cat
0.0629 ± 0.0035,param_moc__cat
0.0415 ± 0.0012,feature_kamera-cofania__cat
0.0384 ± 0.0023,param_skrzynia-biegów__cat
0.0255 ± 0.0038,param_marka-pojazdu__cat
0.0196 ± 0.0014,param_pojemność-skokowa__cat
0.0162 ± 0.0003,feature_bluetooth__cat
